In [1]:
import my_utils.loadModel as lm
import torch

In [2]:
def getBlockList(modelname):
    blocks = []
    if modelname == 'vgg11':
        blocks    = [1, 1, 2, 2, 2]

    if modelname == 'vgg11bn':
        blocks    = [1, 1, 2, 2, 2]

    if modelname == 'vgg13':
        blocks    = [2, 2, 2, 2, 2]

    if modelname == 'vgg13bn':
        blocks    = [2, 2, 2, 2, 2]

    if modelname == 'vgg16':
        blocks    = [2, 2, 3, 3, 3]

    if modelname == 'vgg16bn':
        blocks    = [2, 2, 3, 3, 3]
    return blocks

def getPruneList(modelname):
    prunelist = []
    
    if modelname == 'vgg11':
        prunelist = [0, 3, 6,8, 11,13, 16,18]

    if modelname == 'vgg11bn':
        prunelist = [0, 3, 6,8, 11,13, 16,18]

    if modelname == 'vgg13':
        prunelist = [0,2, 5,7, 10,12, 15,17, 20,22]

    if modelname == 'vgg13bn':
        prunelist = [0,2, 5,7, 10,12, 15,17, 20,22]

    if modelname == 'vgg16':
        prunelist = [0,2, 5,7, 10,12,14, 17,19,21, 24,26,28]

    if modelname == 'vgg16bn':
        prunelist = [0,2, 5,7, 10,12,14, 17,19,21, 24,26,28]
    return prunelist

In [3]:
feature_list = getPruneList('vgg16')
blocks = getBlockList('vgg16')
#getPruneList('vgg16')
print(f"Feature List : {feature_list}")
print(f"Conv layer per Block are :{blocks}")

Feature List : [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]
Conv layer per Block are :[2, 2, 3, 3, 3]


In [4]:
def getPruneModule(newModel,prunelist):
    module = []
    for i in prunelist:
        module.append(newModel.features[i])
    return module

def getPruneCount(module,blocks,maxpr):
    j=0
    count = 0
    prune_prob = []
    prune_count = []
    for i in range(len(module)):
        if(count<blocks[j]):
            frac = 5-j
        else:
            count=0
            j+=1
            frac = 5-j
        prune_prob.append(maxpr/frac)    
        count+=1
    for i in range(len(Module)):
        size = Module[i]._parameters['weight'].shape
        c = int(round(size[0]*prune_prob[i]))
        prune_count.append(c)
    return prune_count
        

In [5]:
model = lm.load_model(model_name='vgg16',number_of_class=6)

prunelist = getPruneList('vgg16')
blocks = getBlockList('vgg16')

Module = getPruneModule(model,prunelist)
prune_count = getPruneCount(module=Module,blocks=blocks, maxpr=.05)

VGG16 Loaded
Linear(in_features=4096, out_features=1000, bias=True)


In [6]:
for i in range(len(Module)):
    print(f"Module {Module[i]} kernel to be prune {prune_count[i]}")

Module Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 1
Module Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 1
Module Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 2
Module Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 2
Module Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 4
Module Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 4
Module Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 4
Module Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 13
Module Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 13
Module Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) kernel to be prune 13
Module Conv2d(512, 512, kernel_size=(3, 3),

In [7]:
print(prunelist)
for i in range(len(model.features)):
    print(f"{i} {model.features[i]}")

[0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]
0 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
1 ReLU(inplace=True)
2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
3 ReLU(inplace=True)
4 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
5 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
6 ReLU(inplace=True)
7 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
8 ReLU(inplace=True)
9 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
10 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
11 ReLU(inplace=True)
12 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
13 ReLU(inplace=True)
14 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
15 ReLU(inplace=True)
16 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
17 Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
18 ReLU(inp

In [8]:
def _compute_distance_score(t, n=1, dim_to_keep=[0,1],threshold=1):
        # dims = all axes, except for the one identified by `dim`        
        dim_to_prune = list(range(t.dim()))   #initially it has all dims
        #remove dim which we want to keep from dimstoprune
        for i in range(len(dim_to_keep)):   
            dim_to_prune.remove(dim_to_keep[i])
        
        size = t.shape
        print(f"\nShape of the tensor: {size}")
        print(f"Print the Dims we want to keep: {dim_to_keep}")
        
        module_buffer = torch.zeros_like(t)
                
        #shape of norm should be equal to multiplication of dim to keep values
        norm = torch.norm(t, p=n, dim=dim_to_prune)
        print(f"norm shape = {norm.shape}")
        size = t.shape
        print("Number Of Features Map in current  layer l     =",size[0])
        print("Number Of Features Map in previous layer (l-1) =",size[1])
        
        for i in range(size[0]):
            for j in range(size[1]):
                module_buffer[i][j] = t[i][j]/norm[i][j]
        
        dist = torch.zeros(size[1],size[0],size[0])
        
        channelList = []
        for j in range(size[1]):
            idxtupple = []
            print('.',end='')
            for i1 in range(size[0]):
                for i2 in range((i1+1),size[0]):
                    dist[j][i1][i2] = torch.norm( (module_buffer[i1][j]-module_buffer[i2][j]) ,p=1)
                    dist[j][i2][i1] = dist[j][i1][i2]
                    
                    if dist[j][i1][i2] < threshold:
                        idxtupple.append([j,i1,i2,dist[j][i1][i2]])
            channelList.append(idxtupple)
        return channelList

In [9]:
#t = Tensor to be prune, n is ln normalization, dim dimension over which we want to perform 
def _compute_kernal_score(t, n=1, dim_to_keep=[0,1],threshold=1):
        # dims = all axes, except for the one identified by `dim`        
        dim_to_prune = list(range(t.dim()))   #initially it has all dims
        #remove dim which we want to keep from dimstoprune
        for i in range(len(dim_to_keep)):   
            dim_to_prune.remove(dim_to_keep[i])
        
        size = t.shape
        print(size)
        print(dim_to_keep)
        
        module_buffer = torch.zeros_like(t)
                
        #shape of norm should be equal to multiplication of dim to keep values
        norm = torch.norm(t, p=n, dim=dim_to_prune)
        return norm

In [10]:
def createPruneListLayerwiseByDistance(startidx,endidx):
    channelTuppleList = []
    st = startidx
    en = endidx
    for i in range(st,en):
        channelTuppleList.append(_compute_distance_score(Module[i]._parameters['weight'],threshold=1))
    return channelTuppleList
channelTuppleList = createPruneListLayerwiseByDistance(0,2)
print("\n\n\nHere is the :",len(channelTuppleList))


Shape of the tensor: torch.Size([64, 3, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 3])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 3
...
Shape of the tensor: torch.Size([64, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 64])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 64
................................................................


Here is the : 2
